- e-commerce의 중요한 부분으로, 고객이 자신과 관계없는 매우 많은 후보 중에 원하는 아이템을 구매하고나 선택할 수 있도록 의사 결정을 도와줌. 수집해 온 이력 데이터를 바탕으로 사용자에게 구매 가능성이 높은 아이템을 제안

- 추천 시스템 분류: 협업 필터링 (Collaborative Filtering), 콘텐츠 기반 필터링 (Content-Based Filtering), 연관 룰 (association rules), 로그 우도 방법 (log-likelyhood method), hybrid methods

- 분석 데이터 세트: [MovieLens](http://grouplens.org/datasets/movielens) : 1,682개의 영화에 대해 943명의 관객이 평가한 100,000개의 영화 평점(1~5점)으로 구성. 각 관객은 최소 20개의 평점을 줌.


[추천 시스템 알고리즘 참고자료](http://bahnsville.tistory.com/897)

# 1. 추천 시스템에서 데이터 집합을 정렬하기 위해 사용하는 주요 행렬과 대표적으로 사용하는 계랑 척도: 유틸리티 행렬, 유사도 척도
## 1) 유틸리티 행렬 

평점 $r_{ij}$은 사용자 $i$가 아이템 $j$를 얼마나 좋게 평가하는지 나타내고 유틸리티 행렬 $R$에 저장됨

행 $i$: 사용자 $i$가 평가한 아이템 목록
열 $j$: 아이템 $j$를 평가한 모든 사용자의 목록

In [1]:
import numpy as np
import pandas as pd
import copy
import collections
from scipy import linalg
import math
from collections import defaultdict

In [2]:
#data
df = pd.read_csv('./data/ml-100k/u.data',sep='\t',header=None)
#movie list
df_info = pd.read_csv('./data/ml-100k/u.item',sep='|',header=None)
movielist = [df_info[1].tolist()[indx]+';'+str(indx+1) for indx in xrange(len(df_info[1].tolist()))]
nmovies = len(movielist)
nusers = len(df[0].drop_duplicates().tolist())  

min_ratings = 50
movies_rated  = list(df[1]) 
counts = collections.Counter(movies_rated)
dfout = pd.DataFrame(columns=['user']+movielist)

toremovelist = []
for i in range(1,nusers):
    tmpmovielist = [0 for j in range(nmovies)]
    dftmp =df[df[0]==i]
    for k in dftmp.index:
        if counts[dftmp.ix[k][1]]>= min_ratings:           
           tmpmovielist[dftmp.ix[k][1]-1] = dftmp.ix[k][2]
           
        else:
           toremovelist.append(dftmp.ix[k][1])
            
    dfout.loc[i] = [i]+tmpmovielist
  
toremovelist = list(set(toremovelist))
dfout.drop(dfout.columns[toremovelist], axis=1, inplace=True)
dfout.to_csv('data/utilitymatrix.csv',index=None)

In [3]:
df = pd.read_csv('data/utilitymatrix.csv')
df.head(2)

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,Richard III (1995);10,...,Cool Runnings (1993);1035,Hamlet (1996);1039,Forget Paris (1995);1041,Multiplicity (1996);1047,She's the One (1996);1048,Koyaanisqatsi (1983);1065,Shallow Grave (1994);1073,Reality Bites (1994);1074,Six Degrees of Separation (1993);1101,Some Kind of Wonderful (1987);1119
0,1.0,5.0,3.0,4.0,3.0,3.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


처음 두 라인 출력한 것을 보면 영화이름;영화ID 로 구성. 평점을 50회 이하로 받은 영화는 유틸리티 행렬에서 제거해 칼럼이 604개.
값이=0: 결측치. 사용자별 또는 아이템별 평균 평점으로 imputation을 해야 할 경우도 있음.

유틸리티 행렬 $R$은 $N$(사용자) X $M$(아이템) 차원.

In [4]:
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in xrange(len(Ri)):
            Ri[i][Ri[i]==0] = sum(Ri[i])/float(len(Ri[i][Ri[i]>0]))
        return Ri
    def itemav():
        for i in xrange(len(Ri[0])):
            Ri[:,i][Ri[:,i]==0] = sum(Ri[:,i])/float(len(Ri[:,i][Ri[:,i]>0]))
        return Ri            
    switch = {'useraverage':userav(),'itemaverage':itemav()}
    return switch[inp]

## 2) 유사도 척도

- 코사인 유사도 (cosine similarity):

<img src="img/eq5-1.cosine.png",width=250,height=250>


- 피어슨 유사도 (Pearson similarity):

<img src="img/eq5-2.pearson.png",width=350,height=250>





두 척도는 평균이 0인 경우에 일치.

sim() 함수는 두 벡터의 유사도를 평가할 때 사용

# 2. 협업 필터링 방법 (Collaborative Filtering methods)

## 사용자는 자신과 비슷한 사람들이 좋게 평가한 아이템을 좋아한다는 아이디어에 기반

- 메모리 기반: 다른 사용자의 기호와 비교해 가장 비슷한 사용자의 기호로 특정 사용자의 평가를 추론

- 모델 기반: 사람들이 좋아하는 아이템에 대한 평가 패턴을 추출하고 이 패턴을 따르는 미래의 평점을 예측

## 특정 사용자를 위한 추천은 데이터에 비슷한 사람들이 얼마나 많이 있는가에 의존하기 때문에 엄청난 양의 데이터를 필요로 함

- Cold start: 예측에 필요한 적정한 크기의 가용 데이터가 없는 상태에서 시작.

- Warm start: 가용 데이터가 충분히 있는 상태로 시작.

- CF와 CBF의 하이브리드 방식: 콜드 스타드를 극복하기 위함.

## 문제점

- 확장성 (scalability) : 사용자와 아이템의 개수에 비례해서 연산이 증가하는 문제점 있음 => 병렬 처리의 필요성

- 희소성 (sparsity) : 사용자가 평가하는 아이템의 개수가 작을 경우 유틸리티 행렬의 희소성이 생김

## 2-1. 메모리 기반 협업 필터링 (Memory-Based Collaborative Filtering)

: 유틸리티 행렬을 이용해 사용자와 아이템 간의 유사도를 계산

: 확장성과 콜트 스타트 이슈

: 많은 사용 시스템에서 유틸리티 행렬이 크거나 아주 작을 경우에 적용


### 2-1-1. 사용자 기반 협업 필터링 (User-Based Collaborative Filtering)

: 현재 사용자와 가거에 비슷한 평가를 사용했던 사용자를 찾기 위해 k-NN 방식을 사용. 이 때 누락 평점을 가중 평균으로 대신 사용.

: 사용자 i와 아직 평가되지 않는 아이템 j에 대해 다음과 같은 단계를 실행한다.

1) 유사도 척도 s를 사용해서 아이템 j에 대해 평점을 줬던 가장 비슷한 사용자 K를 찾는다.

2) 사용자 i가 아직 평가하지 않은 아이템 j에 대해 평점을 예측한다. K의 평점을 가중 평균해 계산한다.

<img src="img/eq5-3.userBasedCF.png",width=350,height=250>

이 때 사용자 i와 k의 평균 평점은 주관적 판단을 보상하기 위해 사용.

균질적인(homogeneous) 평점으로 비교하기 위해 사용자별 평점 분포를 정규화시킬 수 있음.

<img src="img/eq5-4.userBasedCF.png",width=350,height=250>

$\alpha_j$, $\alpha_k$ 는 사용자 i와 k의 평점에 대한 표준 편차. 입력 파라미터 K개의 (20~50) 이웃을 갖는다.

피어슨 상관관계는 상관관계식에서 사용자의 평균 평점을 빼기 때문에 사용자 비교가 쉬우며 코사인 유사도보다 결과가 좋음

In [5]:
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def sim(x,y,metric='cos'):
    if metric == 'cos':
       return 1.-cosine(x,y)
    else:#correlation
       return pearsonr(x,y)[0]

In [4]:
def CF_userbased(u_vec,K,data,indxs=False):
    def FindKNeighbours(r,data,K):
        neighs = []
        cnt=0
        for u in xrange(len(data)):
            if data[u,r]>0 and cnt<K:
               neighs.append(data[u])   
               cnt +=1 
            elif cnt==K:
               break
        return np.array(neighs)
        
    def CalcRating(u_vec,r,neighs):
        rating = 0.
        den = 0.
        for j in xrange(len(neighs)):
            rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
            den += abs(neighs[j][-1])
        if den>0:
            rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:
            rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating>5:
            return 5.
        elif rating<1:
            return 1.
        return rating 
    #add similarity col
    data = data.astype(float)
    nrows = len(data)
    ncols = len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data
    #calc similarities:
    for u in xrange(nrows):
        if np.array_equal(data_sim[u,:-1],u_vec)==False: #list(data_sim[u,:-1]) != list(u_vec):
           data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
           data_sim[u,ncols] = 0.
    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    for r in xrange(ncols):
        if u_vec[r]==0:
            #FindKNeighbours: 사용자 평점 백터인 u_vec과 가장 비슷한 사용자 K를 찾는다
            neighs = FindKNeighbours(r,data_sim,K)
            #CalcRating: Calculate the predicted rating (분포 교정을 하지 않지만 예측 평점>5 or <1인 경우 각각 5,1로 조정)
            u_rec[r] = CalcRating(u_vec,r,neighs)
    if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs] = -1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec    
    return u_rec

### 2-1-2. 아이템 기반 협업 필터링 (Item-Based Collaborative Filtering)

: 사용자가 아닌 아이템에 대해 유사도가 계산된다는 점을 제외하고 개념적으로 사용자 기반 (CF)와 동일

: 사용자 수가 아이템 수보다 매우 커질 경우 아이템 유사도를 사전에 계산함으로써 좀 더 확장성 있는 추천 시스템을 제공. 

알고리즘

1) 유사도 측정치 s를 이용해 사용자 i가 이미 평가한 아이템 중 가장 비슷한 아이템 K개를 찾는다.

2) K개의 아이템 평점을 가중 평균해 예측 평점으로 계산한다.

<img src="img/eq5-5.itemBasedCF.png",width=200>


양수의 $P_{ij}$를 구하기 위해 양수의 유사도만 더하는 것으로 제한 (아이템 평점보다 최고의 아이템을 추천하는데 관심이 있다면 아이템을 상대적으로 정렬)

K = 20~30


In [6]:
class CF_itembased(object):
    def __init__(self,data):
        #calc item similarities matrix
        nitems = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in xrange(nitems):
            for j in xrange(nitems):
                if j>=i:#triangular matrix
                   self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else:
                   self.simmatrix[i,j] = self.simmatrix[j,i]

    # GetKSimItemsperUser: 사용자가 평가하지 않은 아이템과 유사도가 가장 높은 아이템 중 사용자가 과거에 평가했던 K개의 아이템을 찾는다
    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
               neighitems.append(i)
               cnt+=1
            elif cnt==K:
               break
        return neighitems
        
    # CalcRating: 사용자 평점이 없는 경우 예측하며 가중 평균 평점을 계산하여 이웃을 찾을 수 없는 경우 이것으로 설정.
    # u_vec:사용자 평점 벡터, 유틸리티 행렬의 행 벡터
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            rating +=  self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
        
    def CalcRatings(self,u_vec,K,indxs=False):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in xrange(len(u_vec)):
            if u_vec[r]==0:
               neighitems = self.GetKSimItemsperUser(r,K,u_vec)
               #calc predicted rating
               u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs]=-1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec
        return u_rec

#### 2-1-2-1. 슬롭원 (SlopeOne) : 가장 단순한 아이템 기반 협업 필터링

행렬 요소 $d_{ij}$가 아이템 i와 j의 평점 차이 평균인 행렬 D를 계산해보자.

<img src="img/eq5-6.slopeOne.png",width=200>


$n_{k}^{ij}$: 아이템 i와 j에게 평점을 모두 준 사용자를 세는 변수, $\sum_{k=1 }^{N}n_{k}^{ij}$: i와 j에게 평점을 모두 준 사용자 수

알고리즘은 '아이템 기반 협업 필터링' 절에서 설명했던 것과 동일. (차이가 나는 부분은 행렬뿐)

1) 아이템 j와 가장 차가 작은 아이템 K를 찾는다.

2) 예측 평점을 다음의 가중 평균으로 계산한다.

<img src="img/eq5-6-2.slopeOne2.png",width=250>


다른 CF알고리즘에 비해 매우 단순하지만 대체로 정확하고 계산 비용 낮고 구현이 쉬움.

In [6]:
class SlopeOne(object):
    def __init__(self,Umatrix):
        #calc item similarities matrix
        nitems = len(Umatrix[0])
        self.difmatrix = np.zeros((nitems,nitems))
        self.nratings = np.zeros((nitems,nitems))
        def diffav_n(x,y):
            xy = np.vstack((x, y)).T
            xy = xy[(xy[:,0]>0) & (xy[:,1]>0)]
            nxy = len(xy)
            if nxy == 0:
                #print 'no common'
                return [1000.,0]
            return [float(sum(xy[:,0])-sum(xy[:,1]))/nxy,nxy]
            
        # difmatrix: 아이템 i,j의 차 $d_{ij}$를 계산하는데 사용
        for i in xrange(nitems):
            for j in xrange(nitems):
                if j>=i:#triangular matrix                 
                   self.difmatrix[i,j],self.nratings[i,j] = diffav_n(Umatrix[:,i],Umatrix[:,j])
                else:
                   self.difmatrix[i,j] = -self.difmatrix[j,i]
                   self.nratings[i,j] = self.nratings[j,i]
        
    # GetKSimItemsperUser: K개의 최근접 이웃을 찾기 위해 행렬 difmatrix에서 가장 작은 값을 찾는다.
    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.difmatrix[r])
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
               neighitems.append(i)
               cnt+=1
            elif cnt==K:
               break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            if abs(self.difmatrix[r,i])!=1000: #difmatrix 디폴트=1000
               rating +=  (self.difmatrix[r,i]+u_vec[i])*self.nratings[r,i]
               den += self.nratings[r,i]
        if den==0:
            #print 'no similar diff'
            return 0.
        rating = np.round(rating/den,0)
        # 예측 평균이 5보다 크거나 1보다 작으면 각각 5,1로 설정
        if rating >5:
            return 5.
        elif rating <1.:
            return 1.
        return rating
        
    def CalcRatings(self,u_vec,K):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in xrange(len(u_vec)):
            if u_vec[r]==0:
               neighitems = self.GetKSimItemsperUser(r,K,u_vec)
               #calc predicted rating
               u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        return u_rec

## 2-2. 모델 기반 협업 필터링 (Model-Based Collaborative Filtering)

: 유틸리티 행렬을 이용해 사용자와 아이템 평가 패턴을 추출하는 모델을 생성

: 원래의 행렬을 만들거나 근사해 평점을 예측하는 방식

: 행렬 분해 - 유틸리티 행렬로부터 원래의 행렬인 사용자 행렬과 아이템 행렬을 만드는 과정


### 2-2-1. 교대 최소 제곱법 (Alternative Least Square)

행렬 R을 분해하는 가장 간단한 방법. 각 사용자와 아이템은 차원 K의 특징 공간에 표현된다.

<img src="img/eq5-8.utilityMatrix.png",width=140>

여기서 $P(N \times K)$는 특징 공간에서의 새로운 사용자 행렬, $Q(M \times K)$는 특징 공간에서의 아이템 행렬.

<img src="img/eq5-8.ALS.png",width=650>

$\gamma$: 정규화 파라미터, 학습 파라미터인 벡터 $p_i$와 $q_j^T$에 패널티를 주어 값이 커지지 않게 보장함으로써 과적합(overfitting)을 방지.

$Mc_{ij}$는 사용자 i와 아이템 j의 쌍이 실제 평가됐는지 체크. $r_{ij}>0$이면 1, 아니면 0

사용자 벡터 $P_i$와 아이템 벡터 $q_j$에 대해 J를 미분해 0으로 하면 다음 두 식을 얻을 수 있음.

<img src="img/eq5-8.piqj.png",width=300>

교대로 행렬 P, Q를 고정하면 이 식은 최소 제곱 알고리즘 (least square algorithm)으로 직접 풀림.

In [8]:
def ALS(Umatrix, K, iterations=50, l=0.001, tol=0.001): #변수 l은 정규화 파라미터 lamda이며 디폴트로 0.001로 설정

    nrows = len(Umatrix)
    ncols = len(Umatrix[0])  
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    err = 0.
    Umatrix = Umatrix.astype(float)
    mask = Umatrix>0. #행령 Mc는 변수 mask로 정의.
    mask[mask==True]=1
    mask[mask==False]=0
    mask = mask.astype(np.float64, copy=False)
    for it in xrange(iterations):
        for u, mask_u in enumerate(mask):
            #np.linalg.solve로 최소 제곱 문제 풀다
            P[u] = np.linalg.solve(np.dot(Qt, np.dot(np.diag(mask_u), Qt.T)) + l*np.eye(K), 
                                np.dot(Qt, np.dot(np.diag(mask_u), Umatrix[u].T))).T
        for i, mask_i in enumerate(mask.T):
            Qt[:,i] = np.linalg.solve(np.dot(P.T, np.dot(np.diag(mask_i), P)) + l*np.eye(K),
                                np.dot(P.T, np.dot(np.diag(mask_i), Umatrix[:,i])))                            
        err=np.sum((mask*(Umatrix - np.dot(P, Qt)))**2)
        if err < tol:
            break
    return np.round(np.dot(P,Qt),0)

정밀도는 떨어지지만 구현이 매우 쉽고 병렬 처리가 쉬워 속도가 빠르다.

### 2-2-2. 확률 내리막 경사법 (Stochastic Gradient Descent)

[위키피디아 SGD 설명](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) [한글 참고자료](http://darkpgmr.tistory.com/133)

유틸리티 행렬 R을 근사하기 때문에 행렬 분해의 하위 유형에 속함

<img src="img/eq5-8.utilityMatrix.png",width=140>

행렬 P(NXK)와 Q(MXK)는 K차원의 잠재 특징 공간에서의 사용자와 아이템을 나타냄.

근사 평점은 다음과 같이 표현.

행렬 $\widehat{R}$은 정규화된 제곱 오차 $e^2_{ij}$의 최소화 문제를 풀 때 사용. 방법은 ALS와 동일 (3장에서 비용 함수 J).

(p203 중간 수식)


최소화 문제는 내리막 경사법을 이용해 풀 수 있음.

<img src="img/eq5-9.ALS.png",width=300>

$\alpha$는 학습률이고 오류는 $e_{ij}=\left ( r_{ij}-\sum_{K}^{k=l}p_{lk}q_{kj} \right )$임.


두 식을 교대로 적용하면서 수렴할 때까지 R을 찾음.

SGD: 다음절에 나오는 SVD보다 병렬처리가 쉽지만 (처리 속도 빠름) 정밀도가 떨어짐.

In [7]:
def SGD(Umatrix, K, iterations=100, alpha=0.00001, l=0.001, tol=0.001):
    #디폴트: iterations=100, alpha=0.00001, l=0.001, tol=0.001, 평가되지 않은 아이템은 계산에 고려하지 않는다.

    nrows = len(Umatrix)
    ncols = len(Umatrix[0])  
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    cost=-1
    for it in xrange(iterations):
        for i in xrange(nrows):
            for j in xrange(ncols):
                if Umatrix[i][j] > 0:
                   eij = Umatrix[i][j] -np.dot(P[i,:],Qt[:,j])
                   for k in xrange(K):
                       P[i][k] += alpha*(2*eij*Qt[k][j]-l*P[i][k])
                       Qt[k][j] += alpha*(2*eij*P[i][k]-l*Qt[k][j]) 
        cost = 0
        for i in xrange(nrows):
            for j in xrange(ncols):
                if Umatrix[i][j]>0:
                   cost += pow(Umatrix[i][j]-np.dot(P[i,:],Qt[:,j]),2)
                   for k in xrange(K):
                       cost += float(l/2.0)*(pow(P[i][k],2)+pow(Qt[k][j],2))
        if cost < tol:
           break
    return np.round(np.dot(P,Qt),0)

### 2-2-3. 비음수 행렬 분해 (Non-negative Matrix Factorization)

[최적화 방법](https://www.slideshare.net/madvirus/pca-svd)

행렬 R을 두개의 행렬의 곱으로 분해하되 행렬의 요소가 음수가 아닌 방법들의 그룹.

<img src="img/eq5-10.SGD.png",width=150>

$\alpha$는 어떤 정규화 항을 사용할지 정의하는 파라미터. 0: 제곱 정규화, 1: 라소 정규화, 나머지: 두 방법의 혼합

$\gamma$는 정규화 파라미터

In [9]:
from sklearn.decomposition import NMF
def NMF_alg(Umatrix,K,inp='none',l=0.001):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    #imputation
    if inp != 'none':
        R_tmp = imputation(inp,Umatrix)
    nmf = NMF(n_components=K,alpha=l)
    P = nmf.fit_transform(R_tmp) #P는 fit_transform을 통해 구함
    R_tmp = np.dot(P,nmf.components_) #Q^T 행렬은 nmf.components
    return R_tmp

행렬 분해가 되기 전에 대체가 실행 될 수도 있음. 유틸리티 행렬은 양의 값(평점)을 갖기 때문에 유틸리티 행렬 값을 예측하는데 매우 적합

### 2-2-4. 특이값 분해 (Singular Value Decomposition) (기대치 최적화, Expectation Maximization과 함께 적용)

[SVD 위키피디아 설명](http://en.wikipedia.org/wiki/Singular_value_decomposition) [특잇값](https://ko.wikipedia.org/wiki/특잇값)

행렬을 U, $\sigma$, V로 분해해 근사하는 차원 축소 기법 (2장 참조)

초기에 각 사용자별 결측 데이터를 평가할 때 대체가 필요한데, 유틸리티 행렬의 각행의 평균 혹은 열의 평균, 또는 둘의 조합이 사용됨.

유틸리티 행렬에 SVD를 적용하면서 EM 알고리즘(기대치 최대화)을 함께 적용 가능.

1. m-step $\hat{R}=SVD(\hat{})$

2. e-step $\hat{r}_{ij}=\left\{\begin{matrix}
r_{\hat{ij}} \quad if \enspace r_{ij} \enspace is \enspace filled \enspace with \enspace by \enspace the \enspace user\\ 
\hat{r}_{ij} \enspace else \enspace (missing  \enspace data)
\end{matrix}\right.$

제곱 오차의 합이 지정된 허용 오차보다 작아질 때까지 반복한다.

In [10]:
from sklearn.decomposition import TruncatedSVD
# m-step

def SVD(Umatrix,K,inp='none'):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    #imputation
    if inp != 'none':
        R_tmp = imputation(inp,Umatrix)     

    means = np.array([ R_tmp[i][R_tmp[i]>0].mean() for i in xrange(len(R_tmp))]).reshape(-1,1)
    R_tmp = R_tmp-means
    svd = TruncatedSVD(n_components=K, random_state=4)
    R_k = svd.fit_transform(R_tmp)
    R_tmp = svd.inverse_transform(R_k)
    R_tmp = means+R_tmp
    
    return np.round(R_tmp,0)

In [11]:
#e-step
def SVD_EM(Umatrix,K,inp='none',iterations=50,tol=0.001):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    nrows = len(Umatrix)
    ncols = len(Umatrix[0])
    #imputation
    if inp != 'none':
        R_tmp = imputation(inp,Umatrix)
    #define svd
    svd = TruncatedSVD(n_components=K, random_state=4)
    err = -1
    for it in xrange(iterations):
        #m-step
        R_k = svd.fit_transform(R_tmp)
        R_tmp = svd.inverse_transform(R_k)
        #e-step and error evaluation
        err = 0
        for i in xrange(nrows):
            for j in xrange(ncols):
                if Umatrix[i][j]>0:
                   err += pow(Umatrix[i][j]-R_tmp[i][j],2)
                   R_tmp[i][j] = Umatrix[i][j]                   
                   
        if err < tol:
            print it,'toll reached!'
            break
    return np.round(R_tmp,0)

# 3 콘텐트 기반 필터링 방법 (Content-Based Filtering)

아이템의 설명 데이터에 의존해 사용자의 특징을 추출하는 방법.

각 영화는 G 차원의 이진 벡터 $m_j$로 설명됨. 벡터 $m_j$의 요소는 영화 j가 속하는 장르에는 1, 아닌 경우 0.

유틸리티 행렬을 저장한 데이터 프레임 dfout이 주어졌을 때 이진 벡터 $m_j$는 다음의 스크립트를 통해 MovieLens 데이터베이스에서 데이터 프레임으로 수집됨.

In [66]:
#matrix movies's content
movieslist = [int(m.split(';')[-1]) for m in dfout.columns[1:]]
moviescats = ['unknown','Action','Adventure','Animation','Children\'s','Comedy','Crime','Documentary',
              'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
              'Romance','Sci-Fi','Thriller','War','Western']
dfout_movies =  pd.DataFrame(columns=['movie_id']+moviescats)
startcatsindx = 5
cnt= 0
for m in movieslist:
    dfout_movies.loc[cnt] = [m]+df_info.iloc[m-1][startcatsindx:].tolist()
    cnt +=1 
print dfout_movies.head()

dfout_movies.to_csv('data/movies_content.csv',index=None)

   movie_id  unknown  Action  Adventure  Animation  Children's  Comedy  Crime  \
0         1        0       0          0          1           1       1      0   
1         2        0       1          1          0           0       0      0   
2         3        0       0          0          0           0       0      0   
3         4        0       1          0          0           0       1      0   
4         5        0       0          0          0           0       0      1   

   Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  \
0            0      0        0          0       0        0        0        0   
1            0      0        0          0       0        0        0        0   
2            0      0        0          0       0        0        0        0   
3            0      1        0          0       0        0        0        0   
4            0      1        0          0       0        0        0        0   

   Sci-Fi  Thriller  War  Wester

목표: 사용자가 각 장르를 얼마나 좋아하는지를 나타내는 필드를 갖는 사용자 프로파일을 생성하는 것.

단점: 아이템에 대한 콘텐츠 설명 데이터가 늘 존재하지 않아 항상 사용할 수는 없음.

장점: 특정 사용자를 위한 추천이 다른 사용자의 평점과 관련. 아이템에 대한 사용자 평점의 부족으로 발생하는 콜드 스타트 문제는 생기지 않음.

<b>2가지 접근 방식</b>

1. 장르별로 사용자가 본 영화의 평균 평점과 연관된 사용자 프로파일을 생성한 후 코사인 유사도를 사용해 사용자 선호도와 가장 유사한 영화를 찾는다.

2. 정규화된 선형 회귀 분석 모델: 폄점과 영화 특징으로부터 사용자 프로파일 특징을 생성해 사용자가 아직 보지 않은 영화의 평점을 사용자 프로파일을 이용해 예윽

## 3-1. 아이템 특징 평균 방법 (Item Feature Average Method)

In [12]:
class CBF_averageprofile(object):
    def __init__(self,Movies,Movieslist):
        #calc user profiles:
        self.nfeatures = len(Movies[0])
        self.Movieslist = Movieslist 
        self.Movies = Movies
        
    def GetRecMovies(self,u_vec,indxs=False):
        #generate user profile
        nmovies = len(u_vec)
        nfeatures = self.nfeatures
        mean_u = u_vec[u_vec>0].mean()
        diff_u = u_vec-mean_u
        features_u = np.zeros(nfeatures).astype(float)
        cnts = np.zeros(nfeatures)
        for m in xrange(nmovies):
            if u_vec[m]>0:#u has rated m
               features_u += self.Movies[m]*(diff_u[m])
               cnts += self.Movies[m]
        #average:
        for m in xrange(nfeatures):
            if cnts[m]>0:
               features_u[m] = features_u[m]/float(cnts[m])
               
        #calc sim:
        sims = np.zeros(nmovies)
        for m in xrange(nmovies):
            if u_vec[m]==0:#sim only for movies not yet rated by the user
               sims[m] = sim(features_u,self.Movies[m])
        #order movies
        order_movies_indxs = np.argsort(sims)[::-1] 
        if indxs:
            return order_movies_indxs
        return self.Movieslist[order_movies_indxs]

## 3-2. 정규화된 선형 회귀 분석 방법 (Regularized linear regression method)

In [13]:
class CBF_regression(object):
    def __init__(self,Movies,Umatrix,alpha=0.01,l=0.0001,its=50,tol=0.001):
        #calc parameters:
        self.nfeatures = len(Movies[0])+1#intercept
        nusers = len(Umatrix)
        nmovies = len(Umatrix[0])
        #add intercept col
        movies_feats = np.ones((nmovies,self.nfeatures))
        movies_feats[:,1:] = Movies
        self.movies_feats = movies_feats.astype(float)
        
        #set Umatrix as float
        self.Umatrix = Umatrix.astype(float)
        #initialize the matrix:
        Pmatrix = np.random.rand(nusers,self.nfeatures)
        Pmatrix[:,0]=1.
        err = 0.
        cost = -1
        for it in xrange(its):
            print 'it:',it,' -- ',cost
            for u in xrange(nusers):
                for f in xrange(self.nfeatures):                    
                    if f==0:#no regularization
                        for m in xrange(nmovies):
                            if self.Umatrix[u,m]>0:
                               diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                               Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f])
                    else:
                        for m in xrange(nmovies):
                            if self.Umatrix[u,m]>0:
                               diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                               Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f] +l*Pmatrix[u][f])        
                
            cost = 0
            for u in xrange(nusers):
                for m in xrange(nmovies):
                    if self.Umatrix[u][m]>0:
                       cost += 0.5*pow(Umatrix[u][m]-np.dot(Pmatrix[u],self.movies_feats[m]),2)
                for f in xrange(1,self.nfeatures):
                    cost += float(l/2.0)*(pow(Pmatrix[u][f],2))
            if cost < tol:
               print 'err',cost
               break
        self.Pmatrix = Pmatrix
        
    def CalcRatings(self,u_vec):
        #find u_vec
        s = 0.
        u_feats = np.zeros(len(self.Pmatrix[0]))
        #in case the user is not present in the utility matrix find the most similar
        for u in xrange(len(self.Umatrix)):
            #print self.Umatrix[u]
            tmps = sim(self.Umatrix[u],u_vec)
            if tmps > s:
                s = tmps
                u_feats = self.Pmatrix[u]
            if s == 1.:
                break
        new_vec = np.zeros(len(u_vec))
        for r in xrange(len(u_vec)):
            if u_vec[r]==0:
                new_vec[r] = np.dot(u_feats,self.movies_feats[r])
        return new_vec

# 4. 추천 시스템 학습을 위한 연관 규칙

In [15]:
class AssociationRules(object):
    def __init__(self,Umatrix,Movieslist,min_support=0.1,min_confidence=0.1,likethreshold=3):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.Movieslist = Movieslist
        #transform utility matrix to sets of liked items
        nitems = len(Umatrix[0])
        transactions = []
        for u in Umatrix:
            s = [i for i in xrange(len(u)) if u[i]>likethreshold]
            if len(s)>0:
               transactions.append(s)
        #find sets of 2 items
        flat = [item for sublist in transactions for item in sublist]
        inititems = map(frozenset,[ [item] for item in frozenset(flat)])
        set_trans = map(set, transactions)
        sets_init, self.dict_sets_support = self.filterSet(set_trans, inititems)
        setlen = 2
        items_tmp = self.combine_lists(sets_init, setlen)
        self.freq_sets, sup_tmp = self.filterSet(set_trans, items_tmp)
        self.dict_sets_support.update(sup_tmp)
        self.ass_matrix = np.zeros((nitems,nitems))
        for freqset in self.freq_sets:
            #print 'freqset',freqset
            list_setitems = [frozenset([item]) for item in freqset]
            #print "freqSet", freqset, 'H1', list_setitems
            self.calc_confidence_matrix(freqset, list_setitems)
        
    def filterSet(self,set_trans, likeditems):
        itemscnt = {}
        for id in set_trans:
            for item in likeditems:
                if item.issubset(id):
                    itemscnt.setdefault(item, 0)
                    itemscnt[item] += 1
        num_items = float(len(set_trans))
        freq_sets = []
        dict_sets = {}
        for key in itemscnt:
            support = itemscnt[key] / num_items
            if support >= self.min_support:
                freq_sets.insert(0, key)
            dict_sets[key] = support
        return freq_sets, dict_sets
        
    def combine_lists(self,freq_sets, setlen):
        setitems_list = []
        nsets = len(freq_sets)
        for i in range(nsets):
            for j in range(i + 1, nsets):
                setlist1 = list(freq_sets[i])[:setlen - 2]
                setlist2 = list(freq_sets[j])[:setlen - 2]
                if set(setlist1) == set(setlist2):
                    setitems_list.append(freq_sets[i].union(freq_sets[j]))
        return setitems_list
        
    def calc_confidence_matrix(self,freqset, list_setitems):
        for target in list_setitems:
            confidence = self.dict_sets_support[freqset] / self.dict_sets_support[freqset - target]
            if confidence >= self.min_confidence:
                self.ass_matrix[list(freqset - target)[0]][list(target)[0]] = confidence
                
    def GetRecItems(self,u_vec,indxs=False):
        vec_recs = np.dot(u_vec,self.ass_matrix)
        sortedweight = np.argsort(vec_recs)
        seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
        seenmovies = np.array(self.Movieslist)[seenindxs]
        #remove seen items
        recitems = np.array(self.Movieslist)[sortedweight]
        recitems = [m for m in recitems if m not in seenmovies]
        if indxs:
            vec_recs[seenindxs]=-1
            recsvec = np.argsort(vec_recs)[::-1][np.argsort(vec_recs)>0]
            return recsvec
        return recitems[::-1]

# 5. 로그 우도비 추천 시스템 방법

In [14]:
class LogLikelihood(object):
    def __init__(self,Umatrix,Movieslist,likethreshold=3):
        self.Movieslist = Movieslist
        #calculate loglikelihood ratio for each pair
        self.nusers = len(Umatrix)
        self.Umatrix =Umatrix
        self.likethreshold = likethreshold
        self.likerange = range(self.likethreshold+1,5+1)
        self.dislikerange = range(1,self.likethreshold+1)
        self.loglikelihood_ratio()

    def calc_k(self,a,b):
        tmpk = [[0 for j in range(2)] for i in range(2)]
        for ratings in self.Umatrix:
            if ratings[a] in self.likerange and ratings[b] in self.likerange:
               tmpk[0][0] += 1
            if ratings[a] in self.likerange and ratings[b] in self.dislikerange:
                tmpk[0][1] += 1
            if ratings[a] in self.dislikerange and ratings[b] in self.likerange:
                tmpk[1][0] += 1
            if ratings[a] in self.dislikerange and ratings[b] in self.dislikerange:
                tmpk[1][1] += 1
        return tmpk
        
    def calc_llr(self,k_matrix):
        Hcols=Hrows=Htot=0.0
        if sum(k_matrix[0])+sum(k_matrix[1])==0:
            return 0.
        invN = 1.0/(sum(k_matrix[0])+sum(k_matrix[1])) 
        for i in range(0,2):
            if((k_matrix[0][i]+k_matrix[1][i])!=0.0):
               Hcols += invN*(k_matrix[0][i]+k_matrix[1][i])*math.log((k_matrix[0][i]+k_matrix[1][i])*invN )#sum of rows
            if((k_matrix[i][0]+k_matrix[i][1])!=0.0):
               Hrows += invN*(k_matrix[i][0]+k_matrix[i][1])*math.log((k_matrix[i][0]+k_matrix[i][1])*invN )#sum of cols
            for j in range(0,2):
                if(k_matrix[i][j]!=0.0):
                   Htot +=invN*k_matrix[i][j]*math.log(invN*k_matrix[i][j])
        return 2.0*(Htot-Hcols-Hrows)/invN

    def loglikelihood_ratio(self):
        nitems = len(self.Movieslist)
        self.items_llr= pd.DataFrame(np.zeros((nitems,nitems))).astype(float)
        for i in xrange(nitems):
            for j in xrange(nitems):
                if(j>=i):
                   tmpk=self.calc_k(i,j)
                   self.items_llr.ix[i,j] = self.calc_llr(tmpk)
                else:
                   self.items_llr.ix[i,j] = self.items_llr.iat[j,i]
        
    def GetRecItems(self,u_vec,indxs=False):
        items_weight = np.dot(u_vec,self.items_llr)
        sortedweight = np.argsort(items_weight)
        seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
        seenmovies = np.array(self.Movieslist)[seenindxs]
        #remove seen items
        recitems = np.array(self.Movieslist)[sortedweight]
        recitems = [m for m in recitems if m not in seenmovies]
        if indxs:
            items_weight[seenindxs]=-1
            recsvec = np.argsort(items_weight)[::-1][np.argsort(items_weight)>0]
            return recsvec
        return recitems[::-1]

# 6. 하이브리드 추천 시스템

- 가중(Weighted): CF와 CBF 예측 평점을 가중 평균으로 합침

- 혼합(Mixed): CF와 CBF 예측 영화를 각각 구한 후 하나의 목록에 합병

- 교대(Switched): 특정 조건으로 CF 예측 혹은 CBF 예측을 사용

- 특징 조합 (Feature combination): CF와 CBF 특징을 함께 고려해 가장 유사한 사용자나 아이템을 찾음

- 특징 증가 (Feature augmentation): 특징 조합과 유사하지만 추가 특징을 사용해 평점을 요청

In [16]:
class Hybrid_cbf_cf(object):
    def __init__(self,Movies,Movieslist,Umatrix):
        #calc user profiles:
        self.nfeatures = len(Movies[0])
        self.Movieslist = Movieslist 
        self.Movies = Movies.astype(float)
        self.Umatrix_mfeats = np.zeros((len(Umatrix),len(Umatrix[0])+self.nfeatures))
        means = np.array([ Umatrix[i][Umatrix[i]>0].mean() for i in xrange(len(Umatrix))]).reshape(-1,1)
        diffs = np.array([ [Umatrix[i][j]-means[i] if Umatrix[i][j]>0 else 0. 
                            for j in xrange(len(Umatrix[i]))  ] for i in xrange(len(Umatrix))])
        self.Umatrix_mfeats[:,:len(Umatrix[0])] = Umatrix#diffs
        self.nmovies = len(Movies)
        #calc item features for each user
        for u in xrange(len(Umatrix)):
            u_vec = Umatrix[u]
            self.Umatrix_mfeats[u,len(Umatrix[0]):] = self.GetUserItemFeatures(u_vec)
            
    def GetUserItemFeatures(self,u_vec):
        mean_u = u_vec[u_vec>0].mean()
        #diff_u = u_vec-mean_u
        features_u = np.zeros(self.nfeatures).astype(float)
        cnts = np.zeros(self.nfeatures)
        for m in xrange(self.nmovies):
            if u_vec[m]>0:#u has rated m
               features_u += self.Movies[m]*u_vec[m]#self.Movies[m]*(diff_u[m])
               cnts += self.Movies[m]
        #average:
        for m in xrange(self.nfeatures):
            if cnts[m]>0:
               features_u[m] = features_u[m]/float(cnts[m])
        return features_u
    def CalcRatings(self,u_vec,K):
        def FindKNeighbours(r,data,K):
            neighs = []
            cnt=0
            for u in xrange(len(data)):
                if data[u,r]>0 and cnt<K:
                   neighs.append(data[u])   
                   cnt +=1 
                elif cnt==K:
                   break
            return np.array(neighs)
        
        def CalcRating(u_vec,r,neighs):
            rating = 0.
            den = 0.
            for j in xrange(len(neighs)):
                rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
                den += abs(neighs[j][-1])
            if den>0:
                rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
            else:
                rating = np.round(u_vec[u_vec>0].mean(),0)
            if rating>5:
                return 5.
            elif rating<1:
                return 1.
            return rating
        #add similarity col
        nrows = len(self.Umatrix_mfeats)
        ncols = len(self.Umatrix_mfeats[0])
        data_sim = np.zeros((nrows,ncols+1))
        data_sim[:,:-1] = self.Umatrix_mfeats
        u_rec = np.zeros(len(u_vec))
        #calc similarities:
        mean = u_vec[u_vec>0].mean()
        u_vec_feats = u_vec#np.array([u_vec[i]-mean if u_vec[i]>0 else 0 for i in xrange(len(u_vec))])
        u_vec_feats = np.append(u_vec_feats,self.GetUserItemFeatures(u_vec))
        
        for u in xrange(nrows):
            if np.array_equal(data_sim[u,:-1],u_vec)==False: #list(data_sim[u,:-1]) != list(u_vec):
               data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec_feats)
            else:
               data_sim[u,ncols] = 0.
        #order by similarity:
        data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
        #find the K users for each item not rated:
        
        for r in xrange(self.nmovies):
            if u_vec[r]==0:
               neighs = FindKNeighbours(r,data_sim,K)
               #calc the predicted rating
               u_rec[r] = CalcRating(u_vec,r,neighs)
        return u_rec

In [17]:
class Hybrid_svd(object):
    def __init__(self,Movies,Movieslist,Umatrix,K,inp):
        #calc user profiles:
        self.nfeatures = len(Movies[0])
        self.Movieslist = Movieslist 
        self.Movies = Movies.astype(float)
        
        R_tmp = copy.copy(Umatrix)
        R_tmp = R_tmp.astype(float)
        #imputation
        
        if inp != 'none':
            R_tmp = imputation(inp,Umatrix)
        Umatrix_mfeats = np.zeros((len(Umatrix),len(Umatrix[0])+self.nfeatures))
        means = np.array([ Umatrix[i][Umatrix[i]>0].mean() for i in xrange(len(Umatrix))]).reshape(-1,1)
        diffs = np.array([ [float(Umatrix[i][j]-means[i]) 
                            if Umatrix[i][j]>0 else float(R_tmp[i][j]-means[i]) for j in xrange(len(Umatrix[i]))  ] 
                          for i in xrange(len(Umatrix))])
        Umatrix_mfeats[:,:len(Umatrix[0])] = diffs#R_tmp
        self.nmovies = len(Movies)
        #calc item features for each user
        for u in xrange(len(Umatrix)):
            u_vec = Umatrix[u]
            Umatrix_mfeats[u,len(Umatrix[0]):] = self.GetUserItemFeatures(u_vec)
        
        #calc svd
        svd = TruncatedSVD(n_components=K, random_state=4)
        R_k = svd.fit_transform(Umatrix_mfeats)
        R_tmp = means+svd.inverse_transform(R_k)
        self.matrix = np.round(R_tmp[:,:self.nmovies],0)
        
        
    def GetUserItemFeatures(self,u_vec):
        mean_u = u_vec[u_vec>0].mean()
        diff_u = u_vec-mean_u
        features_u = np.zeros(self.nfeatures).astype(float)
        cnts = np.zeros(self.nfeatures)
        for m in xrange(self.nmovies):
            if u_vec[m]>0:#u has rated m
               features_u += self.Movies[m]*(diff_u[m])#self.Movies[m]*u_vec[m]
               cnts += self.Movies[m]
        #average:
        for m in xrange(self.nfeatures):
            if cnts[m]>0:
               features_u[m] = features_u[m]/float(cnts[m])
        return features_u 

# 7. 추천 시스템 평가

In [18]:
def cross_validation(df,k):
    val_num = int(len(df)/float(k))
    print val_num
    df_trains = []
    df_vals = []
    for i in xrange(k):
        start_val = (k-i-1)*val_num
        end_val = start_val+val_num
        df_trains.append(pd.concat([df[:start_val],df[end_val:]]))
        df_vals.append(df[start_val:end_val])

    return df_trains,df_vals

In [19]:
import random
def HideRandomRatings(u_vec, ratiovals=0.5):
    u_test = np.zeros(len(u_vec))
    u_vals = np.zeros(len(u_vec))
    cnt = 0
    nratings = len(u_vec[u_vec>0])
    for i in xrange(len(u_vec)):
        if u_vec[i]>0:        
            if bool(random.getrandbits(1)) or cnt>=int(nratings*ratiovals):
                u_test[i]=u_vec[i]
            else:#random choice to hide the rating:
                cnt +=1
                u_vals[i]=u_vec[i]
    return u_test,u_vals

In [20]:
#load data
df = pd.read_csv('data/utilitymatrix.csv')
print df.head(4)
df_movies = pd.read_csv('data/movies_content.csv')
movies = df_movies.values[:,1:]
print 'check:::',len(df.columns[1:]),'--',len(df_movies)
movieslist = list(df.columns[1:])
#k-fold cv 5 folds
nfolds = 5
df_trains,df_vals = cross_validation(df,nfolds)

   user  Toy Story (1995);1  GoldenEye (1995);2  Four Rooms (1995);3  \
0     1                   5                   3                    4   
1     2                   4                   0                    0   
2     3                   0                   0                    0   
3     4                   0                   0                    0   

   Get Shorty (1995);4  Copycat (1995);5  Twelve Monkeys (1995);7  \
0                    3                 3                        4   
1                    0                 0                        0   
2                    0                 0                        0   
3                    0                 0                        0   

   Babe (1995);8  Dead Man Walking (1995);9  Richard III (1995);10  \
0              1                          5                      3   
1              0                          0                      2   
2              0                          0                      0   
3            

In [22]:
nmovies = len(df_vals[0].values[:,1:][0])
vals_vecs_folds = []
tests_vecs_folds = []
for i in xrange(nfolds):
    u_vecs = df_vals[i].values[:,1:]
    vtests = np.empty((0,nmovies),float)
    vvals = np.empty((0,nmovies),float)
    for u_vec in u_vecs:
        u_test,u_vals = HideRandomRatings(u_vec)
        vvals = np.vstack([vvals,u_vals])
        vtests = np.vstack([vtests,u_test])
    vals_vecs_folds.append(vvals)
    tests_vecs_folds.append(vtests)

## 7.1 평균 제곱근 오차 평가

In [21]:
def SE(u_preds,u_vals):
    nratings = len(u_vals)
    se = 0.
    cnt = 0
    for i in xrange(nratings):
        if u_vals[i]>0:
           se +=  (u_vals[i]-u_preds[i])*(u_vals[i]-u_preds[i])
           cnt += 1
    return se,cnt

In [40]:
err_itembased = 0.
cnt_itembased = 0
err_userbased = 0.
cnt_userbased = 0
err_slopeone = 0.
cnt_slopeone = 0
err_cbfcf = 0.
cnt_cbfcf = 0
for i in xrange(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    cfitembased = CF_itembased(Umatrix)
    cfslopeone = SlopeOne(Umatrix)
    cbfcf = Hybrid_cbf_cf(movies,movieslist,Umatrix)
    print 'fold:',i+1
    vec_vals = vals_vecs_folds[i]
    vec_tests = tests_vecs_folds[i]
    for j in xrange(len(vec_vals)):
        u_vals = vec_vals[j]
        u_test = vec_tests[j]
        #cbfcf
        u_preds = cbfcf.CalcRatings(u_test,5)
        e,c =  SE(u_preds,u_vals)
        err_cbfcf +=e
        cnt_cbfcf +=c
        #cf_userbased
        u_preds = CF_userbased(u_test,5,Umatrix)
        e,c =  SE(u_preds,u_vals)
        err_userbased +=e
        cnt_userbased +=c
        #cf_itembased
        u_preds = cfitembased.CalcRatings(u_test,5)
        e,c =  SE(u_preds,u_vals)
        err_itembased +=e
        cnt_itembased +=c
        #slope one
        u_preds = cfslopeone.CalcRatings(u_test,5)
        e,c =  SE(u_preds,u_vals)
        err_slopeone +=e
        cnt_slopeone +=c
rmse_userbased = np.sqrt(err_userbased/float(cnt_userbased))
rmse_itembased = np.sqrt(err_itembased/float(cnt_itembased))
rmse_slopeone = np.sqrt(err_slopeone/float(cnt_slopeone))
print 'user_userbased rmse:',rmse_userbased,'--',cnt_userbased
print 'user_itembased rmse:',rmse_itembased,'--',cnt_itembased
print 'slope one rmse:',rmse_slopeone,'--',cnt_slopeone

rmse_cbfcf = np.sqrt(err_cbfcf/float(cnt_cbfcf))
print 'cbfcf rmse:',rmse_cbfcf,'---',cnt_cbfcf

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
user_userbased rmse: 1.01381431911 -- 39972
user_itembased rmse: 1.0301785707 -- 39972
slope one rmse: 1.07792084094 -- 39972
cbfcf rmse: 1.0134317593 --- 39972


![table5-1](img/table5-1.png)

In [63]:
err_svd = 0.          
cnt_svd = 0
err_svd_em = 0.
cnt_svd_em = 0
err_als = 0.
cnt_als = 0
err_cbfreg = 0.
cnt_cbfreg = 0
for i in xrange(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    print 'fold:',i+1
    teststartindx = len(Umatrix)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    for k in xrange(len(vals_vecs)):
        u_vals = vals_vecs[k]
        u_test = tests_vecs[k]
        #add test vector to utility matrix
        Umatrix = np.vstack([Umatrix,u_test])
    
    #svd_em_matrix = Hybrid_svd(movies,movieslist,Umatrix,20,'useraverage').matrix#SVD_EM(Umatrix,20,'useraverage',1)
    svd_matrix = SVD(Umatrix,20,'itemaverage')
    cbf_reg = CBF_regression(movies,Umatrix)
    #als_umatrix = SGD(Umatrix,20,50)#ALS(Umatrix,20,50)#NMF_alg(Umatrix,20,'itemaverage',0.001)
    #evaluate errors
    for indx in xrange(len(vals_vecs)):
        #e,c =  SE(als_umatrix[teststartindx+indx],vals_vecs[indx])
        #err_als += e
        #cnt_als += c
        u_preds = cbf_reg.CalcRatings(Umatrix[teststartindx+indx])
        e,c = SE(u_preds,vals_vecs[indx])
        err_cbfreg +=e
        cnt_cbfreg +=c

        e,c = SE(svd_matrix[teststartindx+indx],vals_vecs[indx])
        err_svd +=e
        cnt_svd +=c
        #e,c = SE(svd_em_matrix[teststartindx+indx],vals_vecs[indx])
        #err_svd_em +=e
        #cnt_svd_em +=c

if cnt_svd==0: cnt_svd=1
if cnt_svd_em==0: cnt_svd_em=1
if cnt_als==0: cnt_als=1
if cnt_cbfreg==0: cnt_cbfreg=1

rmse_als = np.sqrt(err_als/float(cnt_als))
rmse_svd = np.sqrt(err_svd/float(cnt_svd))
rmse_svd_em = np.sqrt(err_svd_em/float(cnt_svd_em))
rmse_cbfreg = np.sqrt(err_cbfreg/float(cnt_cbfreg))

print 'svd rmse:',rmse_svd,'--',cnt_svd
#print 'svd_em rmse:',rmse_svd_em,'--',cnt_svd_em
#print 'als rmse:',rmse_als,'--',cnt_als
print 'cbfreg rmse:',rmse_cbfreg,'--',cnt_cbfreg

fold: 1
it: 0  --  -1
it: 1  --  57260.8505699
it: 2  --  47929.3409804
it: 3  --  44144.4291348
it: 4  --  41968.003765
it: 5  --  40493.252468
it: 6  --  39396.671426
it: 7  --  38532.7511847
it: 8  --  37825.4882603
it: 9  --  37230.7281602
it: 10  --  36720.6754199
it: 11  --  36276.7051366
it: 12  --  35885.7066331
it: 13  --  35538.0814244
it: 14  --  35226.5774773
it: 15  --  34945.5739775
it: 16  --  34690.6223712
it: 17  --  34458.1402765
it: 18  --  34245.2004645
it: 19  --  34049.3811689
it: 20  --  33868.6572543
it: 21  --  33701.3193899
it: 22  --  33545.912899
it: 23  --  33401.190744
it: 24  --  33266.0768611
it: 25  --  33139.6372059
it: 26  --  33021.0566321
it: 27  --  32909.6202422
it: 28  --  32804.6982104
it: 29  --  32705.7333291
it: 30  --  32612.2307158
it: 31  --  32523.7492461
it: 32  --  32439.8943805
it: 33  --  32360.3121225
it: 34  --  32284.6839028
it: 35  --  32212.7222259
it: 36  --  32144.1669492
it: 37  --  32078.7820875
it: 38  --  32016.353059
it: 3

![table5-2](img/table5-2.png)

In [ ]:
#user_userbased rmse: 1.01381431911 -- 39972
#user_itembased rmse: 1.0301785707 -- 39972
#slope one rmse: 1.07792084094 -- 39972
#cbfcf rmse: 1.0134317593 --- 39972
#svd rmse: 1.0145666769 -- 39972
#cbfreg rmse: 1.09495415915 -- 39972
#NMF_alg rmse: 0.972259334147 -- 39972
#SVD EM rmse: 1.03845070461 -- 39972
#HYBRID SVD rmse: 1.01385133337 -- 39972
#ALS rmse: 2.58784908254 -- 39972
#SGD rmse: 1.35396020834 -- 39972

In [33]:
def ClassificationMetrics(vec_vals,vec_recs,likethreshold=3,shortlist=50,ratingsval=False,vec_test=None):
    #convert vals in indxs vec
    indxs_like = [i for i in xrange(len(vec_vals)) if vec_vals[i]>likethreshold]
    indxs_dislike = [i for i in xrange(len(vec_vals)) if vec_vals[i]<=likethreshold and vec_vals[i]>0]
    cnt = len(indxs_like)+len(indxs_dislike)
    indxs_rec = []
    if ratingsval:
        #convert ratings into items's list
        if vec_test==None:
            raise 'Error no test vector'
        indxs_rec = [i for i in xrange(len(vec_recs)) if vec_recs[i]>likethreshold and vec_test[i]<1][:shortlist]
    else:
        #consider only the first slot of recs
        indxs_rec = vec_recs[:shortlist]

    tp = len(set(indxs_rec).intersection(set(indxs_like)))
    fp = len(set(indxs_rec).intersection(set(indxs_dislike)))
    fn = len(set(indxs_like)^(set(indxs_rec).intersection(set(indxs_like))))
    precision = 0.
    if tp+fp>0:
        precision = float(tp)/(tp+fp)
    recall = 0.
    if tp+fn>0:
        recall = float(tp)/(tp+fn)
    f1 = 0.
    if recall+precision >0:
        f1 = 2.*precision*recall/(precision+recall)
    
    return np.array([precision,recall,f1]),cnt

![table5-3](img/table5-3.png)

In [61]:
tot_measures = np.zeros(3)    
cnt_vals = 0.
#CF memory based
for i in xrange(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    #cfitembased = CF_itembased(Umatrix)
    #cfslopeone = SlopeOne(Umatrix)
    #cbfcf = Hybrid_cbf_cf(movies,movieslist,Umatrix)
    print 'fold:',i+1
    tot_measures_fold = np.zeros(3)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    for j in xrange(len(vals_vecs)):
        u_vals = vals_vecs[j]
        u_test = tests_vecs[j]
        u_preds = CF_userbased(u_test,20,Umatrix)#cfslopeone.CalcRatings(u_test,5)#cfitembased.CalcRatings(u_test,5)#cbfcf.CalcRatings(u_test,20)
        tmp_measures,cnt_tmp = ClassificationMetrics(u_vals,u_preds,3,50,True,u_test)
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
    tot_measures_fold /= float(len(vals_vecs))
    print tot_measures_fold
    tot_measures += tot_measures_fold
tot_measures /= float(nfolds)
    
print 'precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals

fold: 1
[ 0.56651331  0.16081154  0.23628727]
fold: 2
[ 0.59850305  0.18860111  0.27172641]
fold: 3
[ 0.60989212  0.18333456  0.26413943]
fold: 4
[ 0.60932924  0.19346904  0.27782071]
fold: 5
[ 0.59372861  0.17254562  0.25126217]
precision: 0.595593265581  recall: 0.179752374596  f1: 0.260247197345 --- 39786.0


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [62]:
#CF_userbased precision: 0.595593265581  recall: 0.179752374596  f1: 0.260247197345 --- 39786.0
#CF_itembased precision: 0.573049057653  recall: 0.150154902908  f1: 0.224407731332 --- 39786.0
#SlopeOne precision: 0.572945843878  recall: 0.166998383035  f1: 0.24433916059 --- 39786.0
#Hybrid_cbf_cf precision: 0.600636639987  recall: 0.183293616752  f1: 0.26385405692 --- 39786.0

In [55]:
#CF model based
cnt_vals=0.
tot_measures = np.zeros(3)
for i in xrange(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    print 'fold:',i+1
    teststartindx = len(Umatrix)
    
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    for k in xrange(len(vals_vecs)):
        u_vals = vals_vecs[k]
        u_test = tests_vecs[k]
        #add test vector to utility matrix
        Umatrix = np.vstack([Umatrix,u_test])
    
    #svd_matrix = SVD_EM(Umatrix,20,'useraverage',30)#SVD(Umatrix,20,'itemaverage') #Hybrid_svd(movies,movieslist,Umatrix,20,'useraverage').matrix#SGD(Umatrix,20,50)#ALS(Umatrix,20,50) 
    #matrix=NMF_alg(Umatrix,20,'useraverage')
    #cbf_reg = CBF_regression(movies,Umatrix)
    #cbf_av = CBF_averageprofile(movies,movieslist)
    #llr = LogLikelihood(Umatrix,movieslist)
    assrules = AssociationRules(Umatrix,movieslist)
    
    tot_measures_fold = np.zeros(3)
    for indx in xrange(len(vals_vecs)):
        #u_preds = cbf_reg.CalcRatings(Umatrix[teststartindx+indx])#cbf_av.GetRecMovies(Umatrix[teststartindx+indx],True)
        #u_preds = svd_matrix[teststartindx+indx]#matrix[teststartindx+indx] 
        u_preds = assrules.GetRecItems(Umatrix[teststartindx+indx],True)#llr.GetRecItems(Umatrix[teststartindx+indx],True)
        tmp_measures,cnt_tmp = ClassificationMetrics(vals_vecs[indx],u_preds,3,50,False,Umatrix[teststartindx+indx])
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
    tot_measures_fold = tot_measures_fold/float(len(vals_vecs))
    print tot_measures_fold
    tot_measures += tot_measures_fold
tot_measures = tot_measures/float(nfolds)
print 'precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals

fold: 1
[ 0.64264754  0.29272826  0.37885232]
fold: 2
[ 0.64008348  0.32754908  0.40919527]
fold: 3
[ 0.69438685  0.30352264  0.4023821 ]
fold: 4
[ 0.71752179  0.30733163  0.40810444]
fold: 5
[ 0.70219096  0.3342881   0.42251521]
precision: 0.679366124465  recall: 0.313083943066  f1: 0.404209869025 --- 39786.0


![table5-4](img/table5-4.png)

In [60]:
#llr precision: 0.632059422601  recall: 0.306911656684  f1: 0.389728618382 --- 39786.0
#Hybrid_svd precision: 0.540616355878  recall: 0.122568676777  f1: 0.188867837509 --- 39786.0
#als precision: 0.574768962349  recall: 0.154765744996  f1: 0.232415857722 --- 39786.0
#sgd precision: 0.522492554867  recall: 0.116681592379  f1: 0.182113478188 --- 39786.0
#SVD precision: 0.531278228807  recall: 0.119701346615  f1: 0.184269894611 --- 39786.0
#SVD-EM precision: 0.576567716327  recall: 0.159558142114  f1: 0.236321594653 --- 39786.0
#NMF_alg precision: 0.532487775416  recall: 0.125034210484  f1: 0.191971985488 --- 39786.0
#CBF_regression precision: 0.536374177877  recall: 0.128159010191  f1: 0.196055670058 --- 39786.0
#CBF_averageprofile precision: 0.561491582647  recall: 0.118988755524  f1: 0.185138199893 --- 39786.0
#AssociationRules precision: 0.679366124465  recall: 0.313083943066  f1: 0.404209869025 --- 39786.0